In [6]:
import pandas as pd
import psycopg2
import sqlalchemy as sa

In [20]:
import configparser

def read_config():
    config = configparser.ConfigParser()
    config.read('../config.ini')
    return config

conn_data = read_config()
conn_data = conn_data["PostgreSQL"]

host = conn_data["HOST"]
port = conn_data["PORT"]
database = conn_data["DATABASE"]
user = conn_data["USER"]
password = conn_data["PASSWORD"]
schema = conn_data["SCHEMA"]

'market_data'

In [19]:
def create_companies_table():
    query = f"""
    CREATE TABLE {schema}.companies(
    id_company      SERIAL NOT NULL PRIMARY KEY,
    symbol          VARCHAR(10),
    name            VARCHAR(100)
    );
    """

    with psycopg2.connect(host=host, port=port, user=user, password=password, database=database) as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"DROP TABLE IF EXISTS {schema}.companies;")
            cursor.execute(query)

create_companies_table()

In [16]:
df = 1
table_name = 2

#------------DATABASE CONNECTION---------------------------------
from sqlalchemy.engine.url import URL
# build the sqlalchemy URL
url = URL.create(
drivername='postgresql',
host=host,
port=port,
database=database,
username=user,
password=password,
)

engine = sa.create_engine(url)

#------------LOAD STOCK PRICES DATA---------------------------------
df.to_sql(table_name, con=engine, schema=schema, if_exists='append', index=False, method='multi')
